In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("Speed Duel")

---

Table of Contents <a class="jp-toc-ignore"></a>
=================
* [1 Data aquisition](#data-aquisition)
  * [1.1 Fetch online data](#fetch-online-data)
  * [1.2 Merge data](#merge-data)
* [2 Check changes](#check-changes)
  * [2.1 Load previous data](#load-previous-data)
  * [2.2 Generate changelog](#generate-changelog)
  * [2.3 Save data](#save-data)
* [3 Data visualization](#data-visualization)
  * [3.1 Skill cards](#skill-cards)
    * [3.1.1 Character](#character)
    * [3.1.2 Property](#property)
  * [3.2 Property](#property)
  * [3.3 Archseries](#archseries)
  * [3.4 TCG Speed Duel status](#tcg-speed-duel-status)
      * [3.4.0.1 By card type](#by-card-type)
      * [3.4.0.2 By monster type](#by-monster-type)
      * [3.4.0.3 By archseries](#by-archseries)
    * [3.4.1 TCG Speed Duel vs. TCG status](#tcg-speed-duel-vs.-tcg-status)
* [4 Epilogue](#epilogue)
  * [4.1 HTML export](#html-export)
  <!-- * [4.2 Git](#git) -->

# Data aquisition

## Fetch online data

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
# Fetch skill cards
skill_df = fetch_skill()

# Fetch deck cards
speed_df = fetch_speed()

## Merge data

In [ ]:
full_df = pd.concat([speed_df, skill_df], ignore_index=True, axis=0).sort_values("Name").reset_index(drop=True)
print("Data merged")

# Check changes

## Load previous data

In [ ]:
# Get latest file if exist
tuple_cols = ["Secondary type", "Effect type", "Archseries", "Artwork"]
previous_df, previous_ts = load_latest_data("speed", tuple_cols)

if previous_df is not None:
    previous_df = previous_df.astype(full_df[previous_df.columns.intersection(full_df.columns)].dtypes.to_dict())

## Generate changelog

In [ ]:
if previous_df is None:
    changelog = None
    print("Skipped")
else:
    changelog = generate_changelog(previous_df, full_df, col="Name")
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(
            dirs.DATA / make_filename(report="speed", timestamp=timestamp, previous_timestamp=previous_ts),
            index=True,
        )
        print("Changelog saved")

## Save data

In [ ]:
if changelog is not None and changelog.empty:
    print("No changes. New data not saved")
else:
    full_df.to_csv(
        dirs.DATA / make_filename(report="speed", timestamp=timestamp),
        index=False,
    )
    print("Data saved")

# Data visualization

In [ ]:
full_df

## Skill cards

### Character

In [ ]:
print(
    "Total number of characters portrayed in skill cards:",
    skill_df["Character"].nunique(),
)

In [ ]:
skill_df.drop("Card type", axis=1).sort_values("Character")

In [ ]:
skill_color = plot.colors_dict["Skill Card"]
skill_df["Character"].value_counts().plot.bar(figsize=(18, 6), rot=90, color=skill_color, title="Character")
plt.show()

### Property

In [ ]:
print("Total number of properties:", skill_df["Property"].nunique())

In [ ]:
skill_df.groupby("Property").nunique()

In [ ]:
skill_color = plot.colors_dict["Skill Card"]
skill_df["Property"].value_counts().plot.bar(figsize=(18, 6), rot=0, color=skill_color, title="Property")
plt.show()

## Property

In [ ]:
print("Total number of properties:", full_df["Property"].nunique())

In [ ]:
full_df.groupby("Property").nunique()

In [ ]:
property_unstack = full_df.groupby("Property")["Card type"].value_counts().unstack()
skill_st_colors = [plot.colors_dict[i] for i in property_unstack.columns]
property_unstack.plot.bar(figsize=(14, 6), stacked=True, rot=45, color=skill_st_colors)
plt.show()

## Archseries

In [ ]:
exploded_archseries = full_df.explode("Archseries")
print("Total number of Archseries:", exploded_archseries["Archseries"].nunique())

In [ ]:
exploded_archseries.groupby("Archseries").nunique()

In [ ]:
exploded_archseries["Archseries"].value_counts().head(30).plot.barh(figsize=(10, 15), title="Top 30 Archtypes/Series")
plt.show()

## TCG Speed Duel status

In [ ]:
print("Total number of TCG Speed Duel status:", full_df["TCG Speed Duel status"].nunique())

In [ ]:
full_df.groupby("TCG Speed Duel status", dropna=False).nunique()

In [ ]:
full_df["TCG Speed Duel status"].value_counts(dropna=False).plot.bar(figsize=(18, 6), logy=True, rot=45, title="TCG status")
plt.show()

#### By card type

In [ ]:
# Remove unlimited
tcg_speed_crosstab = pd.crosstab(full_df["Card type"], full_df["TCG Speed Duel status"])
tcg_speed_crosstab

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(
    tcg_speed_crosstab[tcg_speed_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    norm=plot.LogNorm(),
)
plt.show()

#### By monster type

In [ ]:
# Remove unlimited
tcg_speed_crosstab_b = pd.crosstab(full_df["Monster type"], full_df["TCG Speed Duel status"])
tcg_speed_crosstab_b

In [ ]:
plt.figure(figsize=(16, 4))
sns.heatmap(
    tcg_speed_crosstab_b[tcg_speed_crosstab_b > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
)
plt.show()

#### By archseries

In [ ]:
# Remove unlimited
tcg_crosstab_c = pd.crosstab(
    exploded_archseries["Archseries"],
    exploded_archseries["TCG Speed Duel status"],
    margins=True,
)
tcg_crosstab_c

### TCG Speed Duel vs. TCG status

In [ ]:
cg_crosstab = pd.crosstab(full_df["TCG status"], full_df["TCG Speed Duel status"], dropna=False, margins=False)
cg_crosstab

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(
    cg_crosstab[cg_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=plot.LogNorm(),
)
plt.show()

# Epilogue

In [ ]:
benchmark(report="speed", timestamp=timestamp)

In [ ]:
footer()

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
export_notebook(dirs.NOTEBOOKS.user / "Speed.ipynb")

## Git

In [ ]:
git.commit("*[Ss]peed*", f"Speed update - {timestamp.isoformat()}")